# Overview
## Description
A general clustering tool to support non-technical Operations Managers in making informed new market expansion decisions by clustering demand with a built in cost model for new market comparison

## Purpose
Allow dynamic market construction through flexible geographical size while identifying optimally dense clusters or "markets" as expansion candidates

## Product
Output is a series of spreadsheets that can be fed into a Kepler interactive map to facilitate a more intuitive and actionable understanding of latent out of territory demand.

[Example map link](https://drive.google.com/open?id=19Y6ITL2OBuzE_A6xF6qliddHPztV-mjY)

# How to Use
**Watch the tutorial videos: [Part 1](https://www.loom.com/share/25b52af60ff24859bdca26f12c7f5e86) and [Part 2](https://www.loom.com/share/8e76a084e2314faf94995f70be2795a2)**

## What you will need
1. A leads.csv file containing the sales lead data you want to cluster, including at a minimum the following columns: lead_id, zipcode, and status. DO NOT include columns with the names city, state, lat, or lon–these are added automatically. You can optionally include additional columns to include in the map tool tip.
2. An offices.csv file containing the relative efficiencies of each of your current offices/warehouses along with their names and zipcodes
3. A territory.csv file containing all the zipcodes you currently service as in-territory and the office name associated with each one.

## Instructions
1. Make a copy of this iPython notebook and save it in a folder in your google drive
2. Upload the three csv data files to the same folder you created for the copy of this tool
3. Change the data directory in the first cell below to the right folder path you created.
4. Open the "Runtime" menu in this script and click "Run all"
5. Scroll down and watch for cell output or errors. If none, then check your folder for the output files!



---





### Environment Setup
Here you link this tool with the folder or file directory containing your datasets and install necessary dependencies and libraries.

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive/')

#Change to your data directory
os.chdir('/content/drive/My Drive/Example')
#List all files. You should see your lead data in here
!ls '/content/drive/My Drive/Example'

In [0]:
!pip install geopy
!pip install shapely
!pip install uszipcode
!pip install hdbscan

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uszipcode import SearchEngine
from math import cos, asin, sqrt
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import hdbscan
import scipy.cluster as cluster
from scipy.cluster.hierarchy import linkage, dendrogram
import math
from geopy.geocoders import Nominatim
import re
import os
import random

### Zipcode Geocoding
These functions are used for finding the latitude and longitude for each zipcode in your provided datasets, which we will import in the next section. The clean_zip function simply makes sure all the zipcodes have the same format.

In [0]:
#Primary zipcode lookup function: using USZipcode library
def usz_lookup(z):
    search = SearchEngine(simple_zipcode=True)
    zip_dict = search.by_zipcode(z).to_dict()
    return zip_dict['major_city'], zip_dict['state'], zip_dict['lat'], zip_dict['lng']

#Secondary lookup function: using Geopy Nominatim
def nom_lookup(z):
    geolocator = Nominatim(user_agent="_", country_bias='US')
    try:
        location = geolocator.geocode(z, exactly_one=True)
        return location.latitude, location.longitude
    except:
        return None

def geocode_zip(data, drop=True):
    failed_zips = []
    #initialize columns for lookup
    if any(col in data.columns for col in ['city','state','lat','lon']):
      raise NameError('Remove city, state, lat, and lon columns from your leads.csv')
    data = data.reindex(columns = data.columns.tolist() + ['city','state','lat','lon'])
    for z in data['zipcode'].unique():
        #Try primary lookup
        data.loc[data['zipcode']==z,['city','state','lat','lon']] = usz_lookup(z)
        #if no latitude returned, try secondary lookup 
        if pd.isna(data.loc[data['zipcode']==z,'lat']).any():
            data.loc[data['zipcode']==z,['lat','lon']] = nom_lookup(z)
            if pd.isna(data.loc[data['zipcode']==z,'lat']).any():
                failed_zips.append(z)
                data = data.drop(data.index[data['zipcode']==z])
    if len(failed_zips) > 0:
      print('zipcodes dropped (no coordinates): ', failed_zips)
    return data

In [0]:
def clean_zip(data):
    #if all zips are 5 digits, skip this step
    data = data.astype({'zipcode':str})
    if len(data.loc[data['zipcode'].str.len()>5,'zipcode'])==0:
      return data
    #checks for zipcodes with suffixes, replaces them with first 5 digits
    data.loc[data['zipcode'].str.len()>5,'zipcode'] = data['zipcode'].astype(str).apply(lambda x: x[:x.find('-')])
    #adds a leading zero for zipcodes with only 4 digits
    data['zipcode'] = data['zipcode'].str.zfill(5)
    return data

In [0]:
#Input leads data: name file 'leads.csv'. If csv not generated from Microsoft Excel, pass excel = False
#Needs to contain columns: lead_id (unique), zipcode, status
def read_leads(territory, excel=True):
    if excel:
        data = pd.read_csv('leads.csv', encoding = "ISO-8859-1")
    else:
        data = pd.read_csv('leads.csv')
    data = clean_zip(data)
    #checking for zipcodes to drop
    rows = len(data)
    data = data.drop_duplicates(subset='lead_id')
    print('Duplicates dropped: ', rows - len(data))
    rows = len(data)
    data = data.dropna(subset=['zipcode'], how='any')
    print('No-zipcode rows dropped: ', rows - len(data))
    data['serviceable'] = [1 if zp in (territory['zipcode']) else 0 for zp in data['zipcode']]
    data = geocode_zip(data)
    #get job (lead_id) counts per zipcode
    job_counts = data.groupby(['zipcode','status'])['lead_id'].count().reset_index()
    #maps the number of active and cancelled jobs in each zipcode to leads dataframe
    data['zip_active_jobs'] = [job_counts.loc[(job_counts['zipcode']==zp) & (job_counts['status']=='active'),'lead_id'].values[0]\
                               if zp in data.loc[data['status']=='active','zipcode'].values else 0 for zp in data['zipcode'].values]
    data['zip_cancelled_jobs'] = [job_counts.loc[(job_counts['zipcode']==zp) & (job_counts['status']=='cancelled'),'lead_id'].values[0]\
                               if zp in data.loc[data['status']=='cancelled','zipcode'].values else 0 for zp in data['zipcode'].values]
    return data

## Data Import
Here is where your datasets are read by the tool and cleaned using the above functions. If the below cell fails to run, check your file names and the column names in each file to make sure they contain the necessary datapoints.

In [0]:
#Input territory data: name file 'territory.csv'
territory = clean_zip(pd.read_csv('territory.csv'))

#Input leads data: name file 'leads.csv'
leads = read_leads(territory)

#Input office information, reformatting, calculating lat/lon.
offices = geocode_zip(clean_zip(pd.read_csv('offices.csv')))

### Clustering
The **agglomerative** function performs hierarchical complete-linkage clustering on the leads you provided, grouping them into markets based on the size that you specify. The get_cluster_points function adds the cluster label to each lead based on which cluster/market it was grouped into.

Optionally, you can use **HDBSCAN** to cluster your leads instead, which is recommended if the maximum size and shape of the clusters are not a constraint for your business (warehouse hub-and-spoke model not necessary).

In [0]:
def cluster_leads(leads, size, viz=False):
    #complete linkage clustering. Filters out currently serviceable leads (in territory)
    coordinates = np.array([tuple(row) for row in leads.loc[leads['serviceable']==0,['lat','lon']].values])
    #print(len(coordinates[np.isnan(coordinates)]))
    link = linkage(np.radians(coordinates),method='complete')
    #outputs visualization of dendrogram/tree
    if viz:
      print('Drawing dendrogram')
      dend = dendrogram(link)
    #cut the tree, return cluster labels
    cluster_labels = cluster.hierarchy.cut_tree(link, height=size)
    #return number of unique clusters
    cluster_count = len(np.unique(cluster_labels))
    cluster_labels = np.concatenate(cluster_labels)
    #return series of lead coordinates in each cluster
    clusters = pd.Series([coordinates[cluster_labels==n] for n in range(cluster_count)])
    return clusters

In [0]:
#returns dataframe with all the leads in a cluster
def get_cluster_points(clusters):
    cluster_points = pd.DataFrame({'cluster': [x for x in range(len(clusters))],'lat' : [[x[0] for x in row] for row in clusters],\
                              'lon' : [[x[1] for x in row] for row in clusters]})
    #flattening lists
    cluster_points = pd.DataFrame({
        'cluster':np.repeat(cluster_points['cluster'].values, cluster_points['lat'].str.len())}
        ).assign(**{'lat':np.concatenate(cluster_points['lat'].values),
                    'lon':np.concatenate(cluster_points['lon'].values)})#[cluster_points.columns]
    #add cluster to leads df
    leads['cluster'] = leads.apply(lambda row: cluster_points.loc[(cluster_points['lat']==row['lat']) & (cluster_points['lon']==row['lon']),'cluster'].iloc[0], axis=1)
    return leads

In [0]:
def agglomerative(leads):
  #Cut tree for a certain height (h), where h == distance between farthest points in a cluster. Determines cluster size
  miles = int(input('Enter cluster diameter in miles: '))
  #converts miles to radians
  h = miles/3959.
  #warn user of potentially long loading time
  print('Clustering leads now. This may take some time.')
  #series of leads in each cluster
  clusters = cluster_leads(leads, h)
  #add cluster information to leads df
  leads = get_cluster_points(clusters)
  return clusters, leads

In [0]:
def hdensity_based(leads):
  size = int(input('Enter minimum cluster size (number of leads): '))
  clusterer = hdbscan.HDBSCAN(min_cluster_size=size)
  #execute clustering
  print('Clustering leads now. This may take some time.')
  cluster_labels = clusterer.fit_predict(leads[['lat','lon']])
  #cluster count excludes unclustered "noisy" leads
  cluster_count = len(np.unique(cluster_labels))-1
  #series of leads in each cluster
  coordinates = np.array([tuple(row) for row in leads.loc[leads['serviceable']==0,['lat','lon']].values])
  clusters = pd.Series([coordinates[cluster_labels==n] for n in range(cluster_count)])
  #add cluster information to leads df
  leads['cluster'] = cluster_labels
  return clusters, leads

In [0]:
def execute_clustering(leads):
  method = str(input('Enter "a" for agglomerative or "d" for HDBSCAN: '))
  if method == 'a':
    clusters, leads = agglomerative(leads)
    return clusters, leads
  elif method == 'd':
    clusters, leads = hdensity_based(leads)
    return clusters, leads
  else:
    raise(NameError('Invalid method selection: re-run this cell and enter either "a" or "d"'))
  
clusters, test_leads = execute_clustering(leads)

### Mapping to Closest Office

These functions perform a distance calculation from each market's centerpoint to your nearest office, which will be an important input to the cost model and eventual cluster map.

In [0]:
#functions tofind closest office distance and coordinates for each cluster
#geodetic distance calculation
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 2 * 3959 * asin(sqrt(a))

#returns the minimum distance, closest office coordinates, and efficiency
def closest(offices, center):
    dist = np.inf
    for i in range(len(offices)):
        dist2 = distance(center['lat'],center['lon'],offices.loc[i,'lat'],offices.loc[i,'lon'])
        if dist2 < dist:
            dist = dist2
            coords = [offices.loc[i,'lat'], offices.loc[i,'lon']]
            name = offices.loc[i,'office_name']
            eff = offices.loc[i,'efficiency']
    return dist, coords, name, eff

### Creating Map Data Structure
The get_centermost_point function finds the representative or center point of each market to visualize on the map. The make_cluster_map function calculates active and cancelled job counts and creates a data table with each of the cluster centerpoints and their associated metrics.

In [0]:
#build dataframe from cluster output for mapping.
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return centermost_point[0], centermost_point[1]

def make_cluster_map(clusters, offices, cluster_points):
    centroids = clusters.map(get_centermost_point)
    lats, lons = zip(*centroids)
    rep_points = pd.DataFrame({'lon':lons, 'lat':lats, 'closest_distance':'', 'closest_lat':'', \
                               'closest_lon':'', 'office_name': '', 'office_eff':'', 'active_jobs':'',\
                               'cancelled_jobs':''})
    #Calculating active and cancelled job counts    
    job_counts = cluster_points.groupby(['cluster','zipcode'])['zip_active_jobs','zip_cancelled_jobs']\
                    .mean().reset_index()
    for i in range(len(rep_points)):
        center = {'lat' : rep_points.loc[i,'lat'], 'lon': rep_points.loc[i,'lon']}
        x = closest(offices, center)
        rep_points.loc[i,['closest_distance','closest_lat','closest_lon', 'office_name', 'office_eff','active_jobs',\
                          'cancelled_jobs']] = round(x[0],2), x[1][0], x[1][1], x[2], x[3], \
                            job_counts.loc[job_counts['cluster']==i,'zip_active_jobs'].sum(), \
                            job_counts.loc[job_counts['cluster']==i,'zip_cancelled_jobs'].sum()
    return rep_points        

In [0]:
cluster_map = make_cluster_map(clusters, offices, leads)

### Cost Modeling

This section creates the embedded cost model for evaluating each cluster on a per-job cost level. The descriptions for each input are commented in the code below. Make any relevant changes in the section indicated.

In [0]:
#### MODIFY COSTS AND ASSUMPTIONS HERE ####

#Cost model
crew_size = 5 #members per crew
hours = 10 #hours/crew member/day

#Cost Inputs
burden = 1.2 #burden added to operations costs
inhouse_rate = 30 #cost/hour/crew member, taken from efficiency metrics
hotel = 100 #cost/day/crew member, taken from Seattle traveling crew hotel costs
per_diem = 30 #cost/day/crew member
consumables = 200 #cost/job

#Assumptions
flight_thrsh = 500 #distance in miles away from office beyond which crews fly
flight_time = 0.5 #hours per hundred miles
airport_time = 3 #hours spent traveling not on the flight
gas = 0.2 #dollars per mile if driving
driving_speed = 60 #in mph

####

def cost_model(data):
    data = data.reindex(columns = data.columns.to_list()+['days','labor','hotel','travel','perdiem',\
                                           'consumables','cluster_cost','job_cost'])
    for i in range(len(data)):
        jobs = data.loc[i,'active_jobs']
        eff = data.loc[i,'office_eff']
        days = (jobs*eff)/(crew_size*hours)
        labor_cost = inhouse_rate*crew_size*hours*days*burden
        hotel_cost = hotel*crew_size*math.ceil(days)
        distance = data.loc[i,'closest_distance']
        if  distance > flight_thrsh:
            #fly
            flights= crew_size*(50. + (0.11*distance))
            fly_hours = crew_size*inhouse_rate*burden*((flight_time*distance)+airport_time)
            travel_cost = flights + fly_hours
        else:
            #drive
            gas_cost = gas*distance
            drive_hours = crew_size*inhouse_rate*burden*(distance/driving_speed)
            travel_cost = gas_cost+drive_hours
        per_diem_cost = per_diem*crew_size*days
        consumables_cost = consumables*jobs
        if jobs > 0:
            total_cost = sum([labor_cost,hotel_cost,travel_cost,per_diem_cost,consumables_cost])
            job_cost = total_cost/jobs
        else:
            total_cost = 0.
            job_cost = 0.
        #adding costs to data frame
        data.loc[i,['days','labor','hotel','travel','perdiem','consumables','cluster_cost',\
                    'job_cost']] = (days, labor_cost, hotel_cost, travel_cost, per_diem_cost, consumables_cost, round(total_cost,2), round(job_cost,2))
    return data

In [0]:
cost_cluster_map = cost_model(cluster_map)#, cost_inputs())
print(cost_cluster_map)

### Export Model Output
These lines create a csv file for each of the complete clustered leads data table, the offices data table, and the costed cluster mapping table. You can find the files in your google drive working directory that you originally linked colab to in the "Environment Setup" section above

In [0]:
#exporting data tables as csvs for reporting and Kepler
leads.to_csv('cluster_points.csv')
offices.to_csv('geocoded_offices.csv')
cost_cluster_map.to_csv('clusters.csv')

### Data Generation
These function generate the fake datasets used in the training videos. They are included here for reference only.

In [0]:
#Generating Example Data
#Territory data
def territory_gen():
    #lookup function for zipcodes
    search = SearchEngine()
    #Generating territory for a solar contractor based in Los Angeles, California.
    #Services top 25 most populated zipcodes in LA and SF area
    LA_res = search.by_city_and_state('Los Angeles', 'CA', sort_by="population", ascending=False, returns=25)
    territory = pd.DataFrame({'zipcode':[x.zipcode for x in LA_res], 'city':'Los Angeles', 'state':'CA', \
                              'office_name':'LA_branch','office_street':'111 Stuart Ave',\
                              'office_city':'Los Angeles','office_state':'CA','serviceable':1})
    SF_res = search.by_city_and_state('San Francisco', 'CA', sort_by="population", ascending=False, returns=25)
    territory = territory.append(pd.DataFrame({'zipcode':[x.zipcode for x in SF_res], 'city':'San Francisco', 'state':'CA', \
                              'office_name':'SF_branch','office_street':'12 Monte Carlo',\
                              'office_city':'San Francisco','office_state':'CA','serviceable':1}), ignore_index=True)
    return territory
    
#Leads for clustering.
def zip_gen(state):
    search = SearchEngine()
    #unique list of zip codes in a state
    zip_list = set([x.zipcode for x in search.by_state(state, returns=0)])
    #holding list for zip codes of generated jobs
    job_zips = []
    #cap of 1000 leads in each state
    while len(job_zips)<1000:
        zp = random.sample(zip_list,1)[0]
        #up to 30 jobs in each zipcode
        for i in range(random.randint(1,31)):
            job_zips.append(zp)
    return job_zips

def lead_gen(territory):
    #generate zipcodes with artificial demand
    zips = zip_gen('NV') + zip_gen('AZ') + zip_gen('CA') + zip_gen('WA') + zip_gen('OR')
    #generating a lead dataframe of similar structure as the one uploaded
    leads = pd.DataFrame({'lead_id':random.sample(list(range(10000)),len(zips)),'zipcode':zips,\
                  'status':[random.sample(['active','cancelled'],1)[0] for x in range(len(zips))]})
    # leads['serviceable'] = [1 if zp in territory.loc[territory['serviceable']==1,'zipcode'] \
    #                         else 0 for zp in leads['zipcode'].values]
    # #get job (lead_id) counts per zipcode
    # job_counts = leads.groupby(['zipcode','status'])['lead_id'].count().reset_index()
    # leads['zip_active_jobs'] = [job_counts.loc[(job_counts['zipcode']==zp) & (job_counts['status']=='active'),'lead_id'] for zp in leads['zipcode']]
    # leads['zip_cancelled_jobs'] = [job_counts.loc[(job_counts['zipcode']==zp) & (job_counts['status']=='cancelled'),'lead_id'] for zp in leads['zipcode']]
    return leads

def office_gen():
    #minimum columns include office name, zipcode, and efficiency (work hours/job)
    offices = pd.DataFrame({'office_name':['San Francisco','Los Angeles'], 'zipcode':['94102','90089'], 'efficiency':[30,20]})
    return offices

In [0]:
territory = territory_gen()
leads = lead_gen(territory)
offices = office_gen()

#Exporting to csvs
territory.to_csv('territory.csv')
leads.to_csv('leads.csv')
offices.to_csv('offices.csv')